In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import os

In [2]:
## Create netCDF dataset
dataset = Dataset('all_nc.nc', 'w', format='NETCDF4_CLASSIC')

In [3]:
## Dataset parameters
# Misc simulation parameters
nx = 41
ny = 41

# Coordinates
x = dataset.createDimension('x', nx)
y = dataset.createDimension('y', ny)
Time = dataset.createDimension('Time', None)

# Coordinate variables
x_var = dataset.createVariable('x', np.float64, ('x',))
y_var = dataset.createVariable('y', np.float64, ('y',))
Time_var = dataset.createVariable('Time', np.float64, ('Time',))

# Non-coordinate variable
C = dataset.createVariable('C', np.float64, ('Time', 'x', 'y'))

# Variable attributes
x_var.units = 'm'
y_var.units = 'm'
Time_var.units = 's'
C.units = 'kg/m3'

In [4]:
## Fill dataset
# Get number of timesteps
cwd = os.getcwd()
DIR = os.path.join(cwd, 'data', 'csv')
n_files = len([name for name in os.listdir(DIR) 
               if os.path.isfile(os.path.join(DIR, name))])

# Fill limited coordinates
x_var[:] = np.linspace(-1,1,nx)  # Witch magic. You don't overwrite the variable.
y_var[:] = np.linspace(-1,1,ny)

print(C.shape)

# Load csv's
for i in range(n_files):
    fnumber = str(i).zfill(4)    # Zero pad the integer
    fname = 'csv_' + fnumber + '.csv'
    fname = os.path.join('data', 'csv', fname)
    data = np.loadtxt(fname, delimiter=',')

    # Reshape the concentration data
    C_np = np.reshape(data[:,2], (nx,ny))
    
    # Fill netCDF
    C[i,:,:] = C_np
    
Time_var[:] = np.arange(0,n_files,1)
print(C.shape)

(0, 41, 41)
(39, 41, 41)


In [5]:
## Save dataset
dataset.close()